> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# Fundamentos de Estatística com Python

As explicações teóricas e alguns exemplos dessa aula são retiradas do excelente livro de [Fundamentos de estatística para Ciência de Dados (capítulo 2)](https://homepages.dcc.ufmg.br/%7Eassuncao/EstatCC/FECD.pdf) do professor Renato Assunção da UFMG.

---

Vamos fazer uma abordagem prática para introduzir os conceitos básicos de estatística, tomando como base um conjunto de dados real de classificação de mensagens de texto (SMS) como spam ou não-spam.

### A título de curiosidade...
O termo "spam" foi escolhido para indicar mensagens indesejadas com fins comerciais, como uma homenagem a esse quadro de comédia do Monty Python (que também deu nome à linguagem Python). "Spam" também é um alimento enlatado ultraprocessado, que mistura carne de porco com apresuntado (presunto de segunda categoria, mais gorduroso).

[![image alt text](http://img.youtube.com/vi/w7w1MQFLkrE/0.jpg)](http://www.youtube.com/watch?v=w7w1MQFLkrE)

Por isso no dataset a seguir as categorias se chamam `spam` ou `ham` (presunto em inglês), já que o spam seria um presunto de segunda categoria ultraprocessado.

In [ ]:
!wget https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip
!unzip sms+spam+collection.zip

--2024-09-05 17:09:33--  https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘sms+spam+collection.zip’

sms+spam+collection     [ <=>                ] 198.65K  1019KB/s    in 0.2s    

2024-09-05 17:09:34 (1019 KB/s) - ‘sms+spam+collection.zip’ saved [203415]

Archive:  sms+spam+collection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


Cada linha da tabela corresponde a um caso. Casos também são chamados de observações, instâncias, ou exemplos. Cada coluna corresponde a uma variável. Uma variável também é chamada de atributo, ou característica (*feature*, em inglês). No dataset original, cada observação tem dois atributos, o texto do SMS e uma classe binária categorizando o SMS como `spam` ou `ham`.

In [ ]:
import pandas as pd
import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"

df = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Classe', 'SMS'])
df

,Classe,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


O texto é um tipo de **dado não estruturado**, que em termos simples são dados que não seguem um padrão rígido, são mais complexos e mais flexíveis. Por consequência, são mais difíceis de organizar e analisar. Outros exemplos são imagens, vídeos, áudio, etc. **Sua análise geralmente requer a extração de características, transformando o objeto de análise em dados estruturados.**

**Dados estruturados** se definem pela sua estrutura bem-definida, podendo ser organizados em tabelas ou bancos de dados relacionais, facilitando ações como buscas, análises, etc. É sobre esse tipo de dado que falaremos nesse módulo.

Vamos portanto extrair algumas características para usar como base na explicação dos conceitos básicos de estatística:
* `num_chars`: quantidade de caracteres de cada SMS.
* `%alphanum`: define o percentual (entre 0-1) de caracteres alfa-numéricos no SMS.
* `size`: divide as mensagens em três categorias: `small`, `medium` e `large` dependendo do número de caracteres.

In [ ]:
df['num_chars'] = df['SMS'].apply(lambda x: len(x))
df['%alphanum'] = df['SMS'].apply(lambda x: sum([c.isalnum() for c in x])/len(x))
df['size'] = df['num_chars'].apply( lambda x: "small"  if x <= 50 else \
                                              "medium" if x > 50 and x <= 150 else "large")

df

,Classe,SMS,num_chars,%alphanum,size
0,ham,"Go until jurong point, crazy.. Available only ...",111,0.747748,medium
1,ham,Ok lar... Joking wif u oni...,29,0.620690,small
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,0.787097,large
3,ham,U dun say so early hor... U c already then say...,49,0.673469,small
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,0.770492,medium
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,160,0.762500,large
5568,ham,Will ü b going to esplanade fr home?,36,0.777778,small
5569,ham,"Pity, * was in mood for that. So...any other s...",57,0.719298,medium
5570,ham,The guy did some bitching but I acted like i'd...,125,0.792000,medium


## Tipologia de variáveis
Antes de realizar qualquer análise, precisamos primeiro **caracterizar o dado**. Cada tipo de dado permite a aplicação de diferentes análises e exigem cuidados específicos na hora de tratar e interpretar. As variáveis se divivem nos seguintes tipos:

* **Numérica**: Com um variável numérica numa tabela faz sentido somar seus valores (para obter um total geral, por exemplo), subtrair (para medir a diferença entre dois casos, por exemplo) ou tomar médias. Se subdividem em:

    * Numérica **discreta**: Assumem apenas alguns valores com saltos entre eles (geralmente inteiros). A lista de valores possíveis não precisa ser finita, ela precisa ser enumerável. No exemplo a seguir é o atributo `num_chars`.
    * Numérica **contínua**: Seus valores podem assumir qualquer valor num intervalo da reta real. A exemplo do atributo `%alphanum`.
* **Categórica**: Os valores são apenas rótulos indicando diferentes categorias em que os casos podem se classificados. Com variáveis categóricas, em geral, não faz sentido fazer operações aritméticas.
    * Categórica **nominal**: Seus valores possíveis são rótulos de categorias que não podem ser ordenadas. É o caso da variável binária `Classe` que assume um dos dois rótulos `spam` ou `ham`.   
    * Categórica **ordinal**: Cada valor é um rótulo dentre k possíveis categorias, as quais podem ser ordenadas. Existe uma ordem natural nos valores possíveis. Como a variável `size`, cuja ordem de valores estabelece que `small < medium < large`.

## Distribuições

Para encontrar padrões nos dados, e com isso realizar predições ou tomar decisões inteligentes, não trabalhamos com cada observação individual, mas sim com todos os dados da nossa amostra. Isso porque é **a análise do comportamento coletivo que revela tendências**, relações e características que não são visíveis ao se examinar apenas pontos isolados. Olharemos então para a **distribuição** do conjunto de observações, ou seja, a maneira como os valores são organizados ou espalhados. Em termos estatísticos, a distribuição descreve a frequência ou a probabilidade de ocorrência dos diferentes valores ou intervalos de valores em um conjunto de dados.

**Resumos estatísticos, como médias, variâncias e correlações**, desempenham um papel crucial nesse processo. Se usamos a função `describe` do Pandas sobre os dados da nossa tabela, é possível perceber como cada tipo de variável permite uma análise distinta. É sobre essas e outras medidas que falaremos a seguir.



In [ ]:
display(df.describe(include='object'))
display(df.describe(include='number'))

,Classe,SMS,size
count,5572,5572,5572
unique,2,5169,3
top,ham,"Sorry, I'll call later",medium
freq,4825,30,2455


,num_chars,%alphanum
count,5572.000000,5572.000000
mean,80.489950,0.761065
std,59.942907,0.056647
min,2.000000,0.000000
25%,36.000000,0.737416
50%,62.000000,0.769231
75%,122.000000,0.793786
max,910.000000,1.000000


## `statistics`

O Python possui um módulo nativo chamado `statistics`. Como descrito [na documentação](https://docs.python.org/pt-br/3/library/statistics.html), ele não tem intenção de ser uma ferramenta poderosa, mas se igualar ao nível de uma calculadora científica, trazendo os resumos estatísticos mais fundamentais para a análise de distribuições.

### Tendência central ("médias")

As medidas de tendência central mostram os valores centrais ou médios de uma distribuição, ajudando a identificar o valor típico em um conjunto de observações. A média aritmética que aprendemos no ensino básico é só uma das muitas formas de estimar o ponto médio. A tabela a seguir apresenta as funções de média do módulo `statistics`. Em negrito estão alguns que vamos apresentar em seguida.

| Função | Descrição |
| :----- | :-------- |  
| `mean()` |	**Média aritmética.** $(x_1+x_2+...+x_n)/n$ |
| `fmean()` |	Média aritmética otimizada, que sempre retorna `float`. Inclusão opcional de pesos para média ponderada. |
| `geometric_mean()` |	**Média geométrica.**  $\sqrt[n]{x_1*x_2 * ... * x_n}$|
| `harmonic_mean()` |	Média harmônica. |
| `median()` |	**Mediana** (valor central). Se a quantidade de elementos é ímpar, é a média aritmética entre os dois valores centrais. |
| `median_low()` |	Mediana (valor central). Se a quantidade de elementos é ímpar, é o menor entre os dois valores centrais. |
| `median_high()` |	Mediana (valor central). Se a quantidade de elementos é ímpar, é o maior entre os dois valores centrais. |
| `median_grouped()` |	Mediana de dados agrupados em intervalos fixos. Assume distribuição uniforme de elementos dentro de um próprio grupo.  |
| `mode()`  |	Moda (valor mais comum) de dados discretos ou categóricos. |
| `multimode()` |	Lista de modas de dados discretos ou categóricos (em caso de empates). |
| `quantiles()` |	**Quantis** de uma amostra. Divide os dados em intervalos de igual probabilidade. |

Como veremos a seguir, nem toda distribuição é bem comportada, e cada método captura nuances específicas dos dados, garantindo análises mais relevantes ao contexto.

In [ ]:
import statistics as stat

print(f'''fmean() ->\t\tnum_chars: {stat.fmean(df['num_chars'])},
\t\t\t%alphanum: {stat.fmean(df['%alphanum'])}\n''')

print(f'''geometric_mean() ->\tnum_chars: {stat.geometric_mean(df['num_chars'])},
\t\t\t%alphanum: {stat.geometric_mean(df['%alphanum']+1e-4)}\n''')

print(f'''median() ->\t\tnum_chars: {stat.median(df['num_chars'])},
\t\t\t%alphanum: {stat.median(df['%alphanum'])}, ''')

fmean() ->		num_chars: 80.48994974874371,
			%alphanum: 0.7610650854639283

geometric_mean() ->	num_chars: 62.520857231952526,
			%alphanum: 0.7568482347171697

median() ->		num_chars: 62.0,
			%alphanum: 0.7692307692307693, 


In [ ]:
# mediana -> 50% dos valores abaixo da mediana e 50% acima
print(len(df), len(df)//2)
df[df.num_chars <= 62].sort_values('num_chars')
# df[df.num_chars > 62].sort_values('num_chars')

5572 2786


,Classe,SMS,num_chars,%alphanum,size
1925,ham,Ok,2,1.000000,small
4498,ham,Ok,2,1.000000,small
3051,ham,Ok,2,1.000000,small
5357,ham,Ok,2,1.000000,small
5471,ham,Yup,3,1.000000,small
...,...,...,...,...,...
4511,ham,This weekend is fine (an excuse not to do too ...,62,0.790323,medium
1581,ham,"I shall book chez jules for half eight, if tha...",62,0.758065,medium
2011,ham,Dunno lei... I thk mum lazy to go out... I nev...,62,0.645161,medium
893,ham,Nutter. Cutter. Ctter. Cttergg. Cttargg. Ctarg...,62,0.758065,medium


#### Relação entre a média aritmética e a mediana

Veja a figura a seguir, retirada do [tutorial do Real Python](https://realpython.com/python-statistics/#types-of-measures) sobre como descrever seus dados. A média aritmética é sensível a valores extremos. No gráfico, as linhas vermelhas de média são deslocadas para os lados devido aos pontos mais distantes no conjunto de dados. A mediana, por outro lado, é mais estável. Mesmo com a presença de valores extremos (outliers), a mediana permanece centralizada entre os dados principais.

<img src="https://realpython.com/cdn-cgi/image/width=1076,format=auto/https://files.realpython.com/media/py-stats-07.92abf9f362b0.png" height=200>

In [ ]:
fig = df.plot(kind='scatter', y='Classe', x='num_chars')
fig.update_layout(autosize=False,width=700,height=200)
fig.show()

In [ ]:
column = 'num_chars'
fig = df.plot(kind='scatter', y=None, x=column)

fmean = stat.fmean(df[column])
fig.add_trace(go.Scatter(x=[fmean, fmean], y=[0,6000], mode='lines', name=f'fmean'))

geom = stat.geometric_mean(df[column]+1e-4)
fig.add_trace(go.Scatter(x=[geom, geom], y=[0,6000], mode='lines', name=f'geometric'))

median = stat.median(df[column])
fig.add_trace(go.Scatter(x=[median, median], y=[0,6000], mode='lines', name=f'median'))

fig.show()

**Quantis** são valores que dividem um conjunto de dados em partes iguais, ajudando a entender a distribuição dos dados. O mais comum é o **quartil**, que divide os dados em quatro partes iguais, com o primeiro quartil (Q1) marcando o ponto abaixo do qual 25% dos dados estão, o segundo quartil (Q2) sendo a **mediana** (50%), e o terceiro quartil (Q3) marcando o ponto abaixo do qual 75% dos dados estão.

Quantis nos ajudam a entender o conceito de **outlier**, ao destacar os valores que se encontram significativamente fora do **intervalo interquartil** (IQR), que é a diferença entre Q1 e Q3. Valores que estão muito abaixo de Q1 ou muito acima de Q3 (tipicamente 1.5 vezes o IQR para baixo ou para cima) são considerados outliers, indicando dados que são anormalmente baixos ou altos em relação ao restante do conjunto.

> #### Outlier: Se traduz para "ponto fora da curva". São pontos que diferem significativamente da tendência de um conjunto de observações.

In [ ]:
print(f'''quantiles() ->\tnum_chars: {stat.quantiles(df['num_chars'])}''')

df.plot(kind='box', x='num_chars')

quantiles() ->	num_chars: [36.0, 62.0, 122.0]


### Espalhamento, dispersão

A variância (`variance`) e o desvio padrão (*standard deviation* - `stdev`) medem a dispersão dos dados em torno da média. Essas métricas são fundamentais para entender a volatilidade ou consistência de um conjunto de dados. Um baixo desvio padrão indica que os dados estão concentrados em torno da média, enquanto um alto desvio padrão sugere maior dispersão e variabilidade.

| Função | Descrição | ------------------- Fórmula -------------------
| :----- | :-------- | :-- |
| `variance()` | Variância da amostra. | $\sigma^2 = (\sum_{i=1}^N (x_i - \overline{x})^2)/ (N-1)$ |
| `stdev()` |	Desvio padrão da amostra. | $\sigma = \sqrt{(\sum_{i=1}^N (x_i - \overline{x})^2)/ (N-1)}$ |







In [ ]:
print(f'''num_chars ->\t\tfmean()\t\t{stat.fmean(df['num_chars'])},
\t\t\tvariance()\t{stat.variance(df['num_chars'])},
\t\t\tstdev()\t\t{stat.stdev(df['num_chars'])},''')

print('-'*55)

print(f'''%alphanum ->\t\tfmean()\t\t{stat.fmean(df['%alphanum'])},
\t\t\tvariance()\t{stat.variance(df['%alphanum'])},
\t\t\tstdev()\t\t{stat.stdev(df['%alphanum'])},''')

num_chars ->		fmean()		80.48994974874371,
			variance()	3593.1521158115115,
			stdev()		59.942907135135776,
-------------------------------------------------------
%alphanum ->		fmean()		0.7610650854639283,
			variance()	0.003208923511158109,
			stdev()		0.056647361025542124,


Para visualizar a dispersão dos dados usaremos o **histograma**. Um histograma é um gráfico de barras onde cada barra representa um intervalo de valores da distribuição (eixo x), e a altura da barra (eixo y) é a frequência com que os dados caem dentro desse intervalo. Esses intervalos são chamados de bins, e em geral são distribuídos uniformemente (intervalos de tamanho fixo).



In [ ]:
column = 'num_chars'
fig = df.plot(kind='histogram', x=column, marginal="box")

fmean = stat.fmean(df[column])
fig.add_trace(go.Scatter(x=[fmean, fmean], y=[0,300], mode='lines', name=f'fmean'))

std   = stat.stdev(df[column])
fig.add_trace(go.Scatter(x=[fmean+std, fmean+std], y=[0,300], mode='lines', name=f'+stdev'))
fig.add_trace(go.Scatter(x=[fmean-std, fmean-std], y=[0,300], mode='lines', name=f'-stdev'))


## Relações entre duas variáveis

Parte do trabalho de encontrar padrões e tendências nos dados, está em entender a relação entre diferentes atributos. Reconhecer tendências conjuntas, por exemplo, suas notas aumentaram quando você aumenta as horas de estudo, pode indicar que um atributo (horas de estudo) é útil para prever o outro atributo relacionado (suas notas).

Mas note que correlação (relação entre atributos) não indica causalidade (um atributo mudou **por causa** do outro). O site [Spurios Correlations](https://tylervigen.com/spurious/correlation/1199_american-cheese-consumption_correlates-with_googles-annual-global-revenue) é uma excelente maneira de enxergar isso.

<img src="https://tylervigen.com/spurious/correlation/image/1199_american-cheese-consumption_correlates-with_googles-annual-global-revenue.svg" height=250>

<img src="https://tylervigen.com/spurious/correlation/scatterplot/1199_american-cheese-consumption_correlates-with_googles-annual-global-revenue_scatterplot.png" height=250>


Gráficos de dispersão são ótimos para enxergar tendências conjuntas, já que eles apresentam cada ponto (x, y) como uma observação de dois atributos. Ao observar a dispersão de pontos, conseguimos ver se a sua variação segue algum padrão conjunto que pode ser:
* positivo: maiores valores do atributo x correspondem a maiores valores do atributo y (quando um aumenta, o outro aumenta);
* negativo: maiores valores do atributo x correspondem a menores valores do atributo y (quando um aumenta o outro diminui); ou
* correlação fraca: nenhum dos dois padrões está aparente.

A figura a seguir também foi retirada do [tutorial do Real Python](https://realpython.com/python-statistics/#types-of-measures) sobre como descrever seus dados.

<img src="https://realpython.com/cdn-cgi/image/width=1112,format=auto/https://files.realpython.com/media/py-stats-08.5a1e9f3e3aa4.png" height=250>

No módulo `statistics` temos as duas principais funções de correlação:


| Função | Descrição |
| :----- | :-------- |
| `covariance()` | Covariância entre duas variáveis. |
| `correlation()` |	Coeficiente Spearman e Pearson de correlação. |

Ambas as medididas apresentam os resultados:
* Valor positivo para correlação positiva. Quanto maior o valor, mais forte a relação.
* Valor negativo para correlação negativa. Quanto menor o valor (ou maior o valor absoluto), mais forte a relação.
* Valor próximo de zero para correlação fraca.

A covariância tem uma fórmula parecida com a da variância, com a diferença que ela multiplica os termos referentes aos atributos relacionados.

$cov_{X, Y} = \frac{\sum_{i=1}^N (x_i - \overline{x})(y_i - \overline{y})}{(N-1)}$

Já a correlação de Spearman e Pearson (representada pela letra $r$) é calculada em cima do valor de covariância, mas **garantindo que o resultado estará entre $-1 < r < 1$**. Para isso, a covariância é dividida pelo produto dos desvios padrão das variáveis.

$r = cov_{X,Y} / (\sigma_X * \sigma_Y)$

> A correlação traz mais estabilidade à análise já que sabemos que uma correlação forte terá valor absoluto próximo de 1, e correlações fracas estarão muito próximas de 0. Por não ter esses limites tão claros, a covariância é mais difícil de interpretar.

In [ ]:
cov  = stat.covariance( df['num_chars'], df['%alphanum'] )
corr = stat.correlation( df['num_chars'], df['%alphanum'] )

fig = df.plot(kind='scatter', x='num_chars', y='%alphanum')
fig.update_layout(title=f'Covariancia: {cov:.4f}, Correlação: {corr:.4f}')

Como vimos na execução anterior, os atributos `%alphanum`e `num_chars` tem correlação fraquíssima, retornando portanto valores próximo de zero para ambas as métricas. Vejamos agora um outro exemplo com uma base de dados muito utilizada em sala de aula: dados de pacientes com diabetes. Cada observação tem atributos clínicos do paciente, como sua idade, massa corporal, pressão arterial, etc., além de um atributo objetivo (`target`) referente à progressão da diabetes no período de 1 ano.

A [base de dados de diabetes](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html) é uma dos muitos datasets incorporados à biblioteca [Scikit-Learn](https://scikit-learn.org/stable/index.html) do Python.

In [ ]:
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()
diabetes_df = pd.DataFrame(data=diabetes.data,
                           columns=diabetes.feature_names)
diabetes_df['target'] = diabetes.target
display(diabetes_df)

cov  = stat.covariance( diabetes_df['bmi'], diabetes_df['target'] )
corr = stat.correlation( diabetes_df['bmi'], diabetes_df['target']  )

fig = diabetes_df.plot(kind='scatter', x='bmi', y='target')
fig.update_layout(title=f'Covariancia: {cov:.4f}, Correlação: {corr:.4f}')

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0
...,...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207,178.0
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018114,0.044485,104.0
439,0.041708,0.050680,-0.015906,0.017293,-0.037344,-0.013840,-0.024993,-0.011080,-0.046883,0.015491,132.0
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044529,-0.025930,220.0


Podemos simular dados altamente relacionados usando a biblioteca Scikit-Learn. Isso nos dá liberdade de enxergar o comportamento dessas métricas para diferentes tipos de dados. Note como os valores da correlação de Spearman e Pearson são mais fáceis de analisar em relação à covariância, já que o intervalo de correlação é fixo entre -1 e 1.

In [ ]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples=200, n_features=1, random_state=0, noise=40.0)
cov  = stat.covariance( X, y )
corr = stat.correlation( X, y  )

regression_df = pd.DataFrame({'X': X[:,0], 'y': y})
fig = regression_df.plot(kind='scatter', x='X', y='y')
fig.update_layout(title=f'Covariancia: {cov:.4f}, Correlação: {corr:.4f}')

# Referências

* Livro de [Fundamentos de estatística para Ciência de Dados](https://homepages.dcc.ufmg.br/%7Eassuncao/EstatCC/FECD.pdf) (capítulo 2) do professor Renato Assunção da UFMG.
* [Tutorial do Real Python](https://realpython.com/python-statistics/#correlation-coefficient) sobre como descrever seus dados.
* [Documentação](https://docs.python.org/pt-br/3/library/statistics.html#statistics.covariance) do módulo `statistics` do Python.